In [1]:
import torch
from torch import nn
import torchvision
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

import random

import numpy as np

import matplotlib.cm as cm

import pickle

from denoising_assessment_project.utils import functions

from denoising_assessment_project.global_vars import global_vars

%matplotlib notebook

In [2]:
global_vars.init()

In [3]:
# -PARAMETERS FOR GENERATION-

# image dimensions
global_vars.image_dimensions = torch.Size([256,256])

# reps in a series ????
global_vars.series_size = 20

# we want: 
# a spectrum of images 1-50 blobs (50 stacks?) where each image in the series stems from the previous one + 1  
# how do we want to vary the signal? (in the range of 10-1000?), or do this one just for a single signal value?
# how many repetitions per each stack of blob number-signal value (peak value) combination? (previously there were 50)
global_vars.blobs_population_sizes = torch.arange(0,51)
global_vars.peak_values = torch.tensor([200])

In [4]:
print(global_vars.blobs_population_sizes.shape)
print(global_vars.peak_values.shape)

torch.Size([51])
torch.Size([1])


In [5]:
# -IMAGES GENERATION-

# 1. Generating n sets of randomly located blobs for varying quantities of blobs 
# (n = len(blobs_population_sizes))
# dims=[n, reps, img_x, img_y]
distinct_blob_quantities_eval_t2 = functions.varying_blob_quantities_img_series_generate_plus_one(global_vars.series_size, global_vars.blobs_population_sizes, global_vars.image_dimensions)
# tests 
print(distinct_blob_quantities_eval_t2.shape)
print(distinct_blob_quantities_eval_t2.min())
print(distinct_blob_quantities_eval_t2.max())

# 2. Generating m sets of images with different peak values and applying poisson noise to them 
# (m = len(peak_values))
# dims=[2, m, n, reps, img_x, img_y]
# 2a. 
distinct_peaks_and_blob_quantities_eval_t2 = functions.varying_peak_values_apply(distinct_blob_quantities_eval_t2, global_vars.peak_values)
# 2b.
un_noised_images_eval_t2 = functions.background_offset_apply(distinct_peaks_and_blob_quantities_eval_t2, 10)
# 2c.
un_noised_images_eval_t2 = functions.gaussian_beam_apply(un_noised_images_eval_t2)
# 2d.
noised_images_eval_t2 = functions.noise_apply(un_noised_images_eval_t2)

torch.Size([20, 51, 256, 256])
tensor(0.)
tensor(2.1640)


In [6]:
noised_images_eval_t2.shape

torch.Size([1, 20, 51, 256, 256])

In [7]:
# 3. Generating indices for m peak values and n blob quantities reflective of the arragngement of the n x m sets 
# dims=[2, m, n]
indices_eval_t2 = functions.indices_generate_plus_one(noised_images_eval_t2)

In [9]:
print(un_noised_images_eval_t2.shape)
print(noised_images_eval_t2.shape)

print(indices_eval_t2.shape)

print(indices_eval_t2)

torch.Size([1, 20, 51, 256, 256])
torch.Size([1, 20, 51, 256, 256])
torch.Size([2, 1, 51])
tensor([[[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
           14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
           28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
           42,  43,  44,  45,  46,  47,  48,  49,  50]],

        [[200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200,
          200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200,
          200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200,
          200, 200, 200, 200, 200, 200, 200, 200, 200]]])


In [11]:
noised_images_eval_t2_pickled = pickle.dump(noised_images_eval_t2, open("/Users/miriamczech/Desktop/denoising_assessment/pickled_data/EVAL_t2/noised_images_eval_t2.pkl", 'wb'), protocol=4)
%store noised_images_eval_t2_pickled

un_noised_images_eval_t2_pickled = pickle.dump(un_noised_images_eval_t2, open("/Users/miriamczech/Desktop/denoising_assessment/pickled_data/EVAL_t2/un_noised_images_eval_t2.pkl", 'wb'), protocol=4)
%store un_noised_images_eval_t2_pickled

%store indices_eval_t2

Stored 'noised_images_eval_t2_pickled' (NoneType)
Stored 'un_noised_images_eval_t2_pickled' (NoneType)
Stored 'indices_eval_t2' (Tensor)


In [12]:
import os
from tifffile import imwrite

savepath = '/Users/miriamczech/Desktop/denoising_assessment/tiffed_data_beam/EVAL_t2/noised'

for n in range(noised_images_eval_t2.shape[0]):
    for m in range(noised_images_eval_t2.shape[1]):
        for i in range(noised_images_eval_t2.shape[2]):
            im = np.asarray(noised_images_eval_t2[n,m,i,:,:].numpy().astype(np.single))
            
            assert n < 100
            assert m < 100
            assert i < 100
            
            if n in range(0,10):
                n = '0{}'.format(n)
            if m in range(0,10):
                m = '0{}'.format(m)
            if i in range(0,10):
                i = '0{}'.format(i)
            
            filename = '{}_{}_{}.tif'.format(n,m,i)
            
            n = int(n)
            m = int(m)
            i = int(i)

            savename = os.path.join(savepath, filename)
            
            #photometric='minisblack'
            imwrite(savename, im)

n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: 

n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: 

n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: 

n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: 

n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: 

n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: 

n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: 

n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: 

In [13]:
import os
from tifffile import imwrite

savepath = '/Users/miriamczech/Desktop/denoising_assessment/tiffed_data_beam/EVAL_t2/un_noised'

for n in range(un_noised_images_eval_t2.shape[0]):
    for m in range(un_noised_images_eval_t2.shape[1]):
        for i in range(un_noised_images_eval_t2.shape[2]):
            im = np.asarray(un_noised_images_eval_t2[n,m,i,:,:].numpy().astype(np.single))
            
            assert n < 100
            assert m < 100
            assert i < 100
            
            if n in range(0,10):
                n = '0{}'.format(n)
            if m in range(0,10):
                m = '0{}'.format(m)
            if i in range(0,10):
                i = '0{}'.format(i)
            
            filename = '{}_{}_{}.tif'.format(n,m,i)
            
            n = int(n)
            m = int(m)
            i = int(i)

            savename = os.path.join(savepath, filename)
            
            #photometric='minisblack'
            imwrite(savename, im)

n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: 

n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: 

n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: 

n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: 

n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: 

n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: 

n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: 

n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: <class 'int'>
n datatype: <class 'int'>
m datatype: <class 'int'>
i datatype: 

7
